# I90 Settlement Files

I90DIA files contain detailed per-unit settlement data for the Spanish
electricity market. This notebook shows how to download and parse them.

In [1]:
from esios import ESIOSClient

client = ESIOSClient()

## Download I90DIA files

Archive ID 34 = I90DIA (daily settlement by programming unit).

In [2]:
path = client.archives.download(
    34,
    date="2024-06-15",
    output_dir="data",
)
print(f"Extracted to: {path}")

Extracted to: /Users/sota/.cache/esios/archives/34/I90DIA_20240615


## Parse with I90Book

The `I90Book` class reads the XLS file and gives access to individual sheets:

In [3]:
from pathlib import Path
from esios.processing.i90 import I90Book

# Find the XLS file
xls_files = list(Path("data").rglob("I90DIA*.xls"))
print(f"Found: {[f.name for f in xls_files]}")

Found: ['I90DIA_20240615.xls']


In [4]:
book = I90Book(xls_files[0])
print(f"Date: {book.metadata['date_data']}")
print(f"Sheets: {list(book.table_of_contents.keys())[:10]}")

Date: 2024-06-15 00:00:00
Sheets: ['I90DIA00', 'I90DIA01', 'I90DIA02', 'I90DIA03', 'I90DIA04', 'I90DIA05', 'I90DIA06', 'I90DIA07', 'I90DIA08', 'I90DIA09']


## Read a specific sheet

Sheet "I90DIA03" contains generation by technology:

In [5]:
sheet = book["I90DIA03"]
df = sheet.df
print(f"Frequency: {sheet.frequency}")
df

Frequency: hourly


value
Redespacho Sentido Unidad de Programación Nm Oferta asignada Tipo Oferta Tipo cálculo Tipo Restricción datetime                        
UPOPVPV    Subir   ABO2                   32162900           1.0         Complejo     RTD              2024-06-14 22:00:00+00:00    NaN
                                                                                                       2024-06-14 23:00:00+00:00    NaN
                                                                                                       2024-06-15 00:00:00+00:00    NaN
                                                                                                       2024-06-15 01:00:00+00:00    NaN
                                                                                                       2024-06-15 02:00:00+00:00    NaN
...                                                                                                                                 ...
ECOCB      Bajar   UPVLEBR                26112057           10.0        No aplica    N/A              2024-06-15 17:00:00+00:00    NaN
                                                                                                       2024-06-15 18:00:00+00:00    NaN
                                                                                                       2024-06-15 19:00:00+00:00    NaN
                                                                                                       2024-06-15 20:00:00+00:00    NaN
                                                                                                       2024-06-15 21:00:00+00:00    NaN

[5136 rows x 1 columns]

## Export processed data

In [6]:
df.to_parquet("data/i90_sheet_03.parquet")